
# Parte 1: Análisis Exploratorio de Datos (EDA)
## Carga de datos

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_orig = pd.read_csv("./datos/netflix_originals.csv", index_col=0)
df_titles = pd.read_csv("./datos/netflix_titles.csv", index_col=0)

## 1.1 Primero analizaremos df_orig (netflix_originals.csv):
Esta es la estructura que seguiré
1. Análisis inicial de la estructura
2. Análisis de variables categóricas y numéricas
3. Preparación para la unión

In [4]:
print("Ejemplo del DataFrame:")
df_orig.head()

Ejemplo del DataFrame:


,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian
3,The Open House,Horror thriller,"January 19, 2018",94,3.2,English
4,Kaali Khuhi,Mystery,"October 30, 2020",90,3.4,Hindi


In [5]:
print("Información básica del DataFrame:")
df_orig.info()

Información básica del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 0 to 583
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       513 non-null    object 
 1   Genre       513 non-null    object 
 2   Premiere    513 non-null    object 
 3   Runtime     513 non-null    int64  
 4   IMDB Score  513 non-null    float64
 5   Language    513 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 28.1+ KB


In [6]:
df_orig.describe(include=['object', 'category'])

,Title,Genre,Premiere,Language
count,513,513,513,513
unique,513,106,350,37
top,Enter the Anime,Documentary,"November 1, 2019",English
freq,1,132,5,352


## 1.1.1 Análisis inicial de la estructura:

- 513 Registros pero el índice más alto es 584, esto se puede deber a que los índices no sean secuanciales y haya algún gap.
- No contamos con valores nulos
- La columna `Premiere`, aunque indique una fecha, es de tipo String
- Los 513 regiustros se distribuyen en 106 género diferentes
- Casi el 70% del contenido está en Inglés
- El género más común es el documental

In [7]:
for columna in ['Title', 'Genre', 'Language']:
    print(f"Número de valores únicos en la columna {columna}: {df_orig[columna].nunique()}")
    

Número de valores únicos en la columna Title: 513
Número de valores únicos en la columna Genre: 106
Número de valores únicos en la columna Language: 37


## 1.1.2 Análisis variable snuméricas y categóricas:

In [8]:
resultados = df_orig.describe().round(2)
resultados

,Runtime,IMDB Score
count,513.00,513.00
mean,94.67,6.21
std,26.65,0.97
min,4.00,2.50
25%,87.00,5.60
50%,97.00,6.30
75%,108.00,6.90
max,209.00,9.00


### Para Runtime (duración en minutos)
- Un amplio rango desde 4  a 209, esto viene dado por la variedad de contenido (cortometraje, documental...)
- Cuenta con una media de 94, apenas diferente a la mediana en 97, lo que nos indica una distribución equilibrada
### Para IMDB Score (medida de puntuación)
- Similar a la anterior, también cuenta con una distribución equilibrada y rango de datos entre 2.5 y 9

`En base a estos será interesante estudiar cómo se ve afectada la puntuación con respecto a otras variables cómo género o duración mismamente.`



## 1.1.3 Preparación para la unión:
- Ver si existen títulos duplicados
- Cambiar el formato de la columna `Premiere` a tipo fecha
- A partir del nuevo formato de `Premiere`, crear una nueva columna `release_year`que nos sirva para comparar resultados con el otro df

In [10]:
print(df_orig['Title'].duplicated().sum())

0


Por tanto confirmamos, que todos los registros son únicos, tomando como referencia el título.

In [11]:
df_orig['Premiere'] = df_orig['Premiere'].str.replace('.', ',')
df_orig['Premiere'] = pd.to_datetime(df_orig['Premiere'])
df_orig['release_year'] = df_orig['Premiere'].dt.year

### Ejemplo de la estructura actualizada
Ahora `Premiere`es de tipo date y contamos con una nueva columna `release_year`

In [12]:
df_orig.head()

,Title,Genre,Premiere,Runtime,IMDB Score,Language,release_year
0,Enter the Anime,Documentary,2019-08-05,58,2.5,English/Japanese,2019
1,Dark Forces,Thriller,2020-08-21,81,2.6,Spanish,2020
2,The App,Science fiction/Drama,2019-12-26,79,2.6,Italian,2019
3,The Open House,Horror thriller,2018-01-19,94,3.2,English,2018
4,Kaali Khuhi,Mystery,2020-10-30,90,3.4,Hindi,2020


### 1.1.3 Breve análisis temporal
Podemos ver un incremeneto año por año, hasta en 2021 donde contamos con menos registros.

In [13]:
print(df_orig['release_year'].value_counts().sort_index())

release_year
2014      1
2015      9
2016     28
2017     57
2018     85
2019    107
2020    164
2021     62
Name: count, dtype: int64


## Resumen sobre netflix_original.csv

- 513 Registros
- No hay valores duplicados
- Los 513 registros se distribuyen en 106 género diferentes, a lo largo de 7 años
- Casi el 70% del contenido está en Inglés
- El género más común es el documental
- La puntuación media es de 6,2, sinedo el máximo 9 y el mínimo 2.5
- La duracion media está en 94 minutos

----

## 1.2 Análisis de df_titles (netflix_titles.csv):
Esta es la estructura que seguiré
1. Análisis inicial de la estructura
2. Análisis de valores faltantes
3. Análisis de variables categóricas
4. Preparación para la unión

In [14]:
print("Ejemplo del DataFrame:")
df_titles.head()

Ejemplo del DataFrame:


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,NaN,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,NaN,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [15]:
print("Información básica del DataFrame:")
df_titles.info()

Información básica del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      3994 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


## 1.2.1 Análisis inicial de la estructura:
- Contamos con 8807 registros y 12 columnas.
- Sobre el tipo de las columnas, a excepción de release_year que es un número entero, el resto son Strings
- En este caso, a diferencia del dataframe anterior, si que contamos con bastantes nulos. 

In [16]:
df_titles.describe(include=['object', 'category'])

,show_id,type,title,director,cast,country,date_added,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8803,3994,8807,8807
unique,8807,2,8807,4528,7692,748,1767,17,185,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",TV-MA,2 Seasons,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,3207,425,362,4


### Continuando con el análisis inicial:
- No contamos con registros duplicados
- 4528 Directores únicos
- Divididos en 748 paises
- Solo hay 2 tipos de contenidos

## 1.2.2 Análisis de los valores faltantes:


In [17]:
nulos = df_titles.isnull().sum()
porcentajeNulos = ((df_titles.isnull().sum() / len(df_titles)) * 100).round()

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos,
    'Porcentaje': porcentajeNulos
})

resumen_nulos = resumen_nulos[resumen_nulos['Valores Nulos'] > 0]
print(resumen_nulos)

            Valores Nulos  Porcentaje
director             2634        30.0
cast                  825         9.0
country               831         9.0
date_added             10         0.0
rating                  4         0.0
duration             4813        55.0


Podemos ver porcentajes preocupantes de datos faltantes en columnas como `duration`y `director`y en menor proporción el `country`y `cast`.


## 1.2.3 Análisis de las variables categóricas:

In [18]:
porcentaje_type = (df_titles['type'].value_counts(normalize=True) * 100).round()
print("En porcentajes:")
print(porcentaje_type)

En porcentajes:
type
Movie      70.0
TV Show    30.0
Name: proportion, dtype: float64


In [87]:
generos = df_titles['listed_in'].str.split(',').explode().str.strip()
porcentaje_generos = (generos.value_counts(normalize=True) * 100).round()

print("Distribución por géneros en porcentages")
print(porcentaje_generos.head(10))

Distribución por géneros en porcentages
listed_in
International Movies        14.0
Dramas                      13.0
Comedies                     9.0
International TV Shows       7.0
Documentaries                4.0
Action & Adventure           4.0
TV Dramas                    4.0
Independent Movies           4.0
Children & Family Movies     3.0
Romantic Movies              3.0
Name: proportion, dtype: float64


## 1.2.4 Preparación para la unión
1. Cambiar el tipo de la columna `date_added` para que sea de tipo date.
2. Hacer coincidir la columna en común, en este caso `title`-> `Title`

In [19]:
df_titles['date_added'] = df_titles['date_added'].str.strip()
df_titles['date_added'] = pd.to_datetime(df_titles['date_added'])

In [20]:
print("Nuevo formato:")
print(df_titles['date_added'].head())
print("Tipo de dato de la columna date_added:")
print(df_titles['date_added'].dtype)

Nuevo formato:
0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]
Tipo de dato de la columna date_added:
datetime64[ns]


In [21]:
df_titles.rename(columns={'title': 'Title'}, inplace=True)

## 2 Unión de datos
Antes de proceder a la unión de ambos dataframes, quiero verificar que están escritos de la misma manera, viendo si coinciden.
`Como podemos ver tódos los títulos originales se encuentran en netflix_titles.`

In [22]:
#Para hacer esta comparación hago uso de los sets, que nos facilitan ver sus datos comunes
titulos_comunes = set(df_orig['Title']).intersection(set(df_titles['Title']))
print("Número de títulos que coinciden exactamente:", len(titulos_comunes))

Número de títulos que coinciden exactamente: 513


Teniendo esto en cuenta, usaré un 'left merge', así mantenemos todos los registros de netflix_titles (catálogo completo) y añadimos la información adicional de netflix_originals donde haya coincidencia. Como podemos ver el dataset final conserva la longitud de netflix_titles como esperábamos.

In [23]:
df_resultante = df_titles.merge(df_orig, on='Title', how='left', indicator=True)

print("Número total de filas en el dataset final:", len(df_resultante))

Número total de filas en el dataset final: 8807


In [24]:
df_resultante.sample()

,show_id,type,Title,director,cast,country,date_added,release_year_x,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,release_year_y,_merge
793,s794,Movie,Fools Rush In,Andy Tennant,"Matthew Perry, Salma Hayek, Jon Tenney, Carlos...",United States,2021-06-02,1997,PG-13,NaN,"Comedies, Romantic Movies","When a one-night stand results in a pregnancy,...",NaN,NaT,NaN,NaN,NaN,NaN,left_only


In [25]:
# Al hacer el merge, si ambos dataframes cuentan con columnas con el mimo título, 
# se le cambia el título añadiendo una x para indicar la que viene por la izquierda
# e y para la derecha. 
# Para evitarlo, o cambiar el nombre o añadir el atributo suffix?
df_resultante = df_resultante.rename(columns={'release_year_x': 'release_year'})
df_resultante = df_resultante.drop('release_year_y', axis=1)

## 3 Uso de Loc e iLoc
## Ejercicios de Loc

In [33]:
# 1.1 Películas
peliculas = df_resultante.loc[df_resultante['type'] == 'Movie']
print("1.1 Número de películas:", len(peliculas))

# 1.2 Series de TV con título y duración
series_tv = df_resultante.loc[df_resultante['type'] == 'TV Show', ['Title', 'duration']]
print(f"\n1.2 Series de TV con título y duración: {len(series_tv)}")
print(series_tv.head(3))

# 1.3 Contenido de Estados Unidos
contenido_eeuu = df_resultante.loc[df_resultante['country'] == 'United States']
print("\n1.3 Contenido de Estados Unidos:", len(contenido_eeuu))

# 1.4 Películas con título y director
peliculas_director = df_resultante.loc[df_resultante['type'] == 'Movie', ['Title', 'director']]
print("\n1.4 Películas con título y director:", len(peliculas_director))
print(peliculas_director.head(3))

# 1.5 Contenido de 2018
contenido_2018 = df_resultante.loc[df_resultante['release_year'] == 2018, ['Title', 'listed_in']]
print("\n1.5 Contenido de 2018:")
print(contenido_2018.head(3))

# 1.6 Títulos sin director
sin_director = df_resultante.loc[df_resultante['director'].isnull(), ['Title']]
print("\n1.6Títulos sin director:")
print(sin_director.head(3))

1.1 Número de películas: 6131

1.2 Series de TV con título y duración: 2676
                   Title   duration
1          Blood & Water  2 Seasons
2              Ganglands        NaN
3  Jailbirds New Orleans        NaN

1.3 Contenido de Estados Unidos: 2818

1.4 Películas con título y director: 6131
                              Title                       director
0              Dick Johnson Is Dead                Kirsten Johnson
6  My Little Pony: A New Generation  Robert Cullen, José Luis Ucha
7                           Sankofa                   Haile Gerima

1.5 Contenido de 2018:
                     Title                                          listed_in
21  Resurrection: Ertugrul  International TV Shows, TV Action & Adventure,...
37             Angry Birds                              Kids' TV, TV Comedies
73            King of Boys                       Dramas, International Movies

1.6Títulos sin director:
                   Title
1          Blood & Water
3  Jailbirds New O

## Ejercicios de iLoc

In [42]:
df_resultante.sample()

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge
6972,s6973,Movie,Hiroshima: The Real History,Lucy van Beek,John Sessions,United Kingdom,2017-03-31,2015,TV-PG,NaN,Documentaries,This detailed examination of the 1945 Hiroshim...,NaN,NaT,NaN,NaN,NaN,left_only


In [45]:
# Primeras 5 filas mostrando title, director y country.
primeras_cinco = df_resultante.iloc[0:5, [2, 3, 5]]
print("Primeras 5 filas, sólo mostrando las columnas title, director y country")
print(primeras_cinco)

# Últimas 5 filas con title y description
ultimas_cinco = df_resultante.iloc[-5:, [2, 11]]
print("\nÚltimas 5 filas con título y descripción:")
print(ultimas_cinco)

# Primeras 10 filas, columnas 2-5
primeras_diez = df_resultante.iloc[0:10, 1:5]
print("\nPrimeras 10 filas, columnas 2-5:")
print(primeras_diez)

# Últimas 7 filas con title, director y country
ultimas_siete = df_resultante.iloc[-7:, [2, 3, 5]]
print("\nÚltimas 7 filas, title, director y country:")
print(ultimas_siete)

# Filas 15-25, primeras 4 columnas
filas_15_25 = df_resultante.iloc[15:26, 0:4]
print("\nFilas 15-25, primeras 4 columnas:")
print(filas_15_25)

# Filas 20-30 con title, release_year y rating
filas_20_30 = df_resultante.iloc[20:31, [2, 7, 8]]
print("\nFilas 20-30, title, release_year y rating:")
print(filas_20_30)

Primeras 5 filas, sólo mostrando las columnas title, director y country
                   Title         director        country
0   Dick Johnson Is Dead  Kirsten Johnson  United States
1          Blood & Water              NaN   South Africa
2              Ganglands  Julien Leclercq            NaN
3  Jailbirds New Orleans              NaN            NaN
4           Kota Factory              NaN          India

Últimas 5 filas con título y descripción:
            Title                                        description
8802       Zodiac  A political cartoonist, a crime reporter and a...
8803  Zombie Dumb  While living alone in a spooky town, a young g...
8804   Zombieland  Looking to survive in a world taken over by zo...
8805         Zoom  Dragged from civilian life, a former superhero...
8806       Zubaan  A scrappy but poor boy worms his way into a ty...

Primeras 10 filas, columnas 2-5:
      type                             Title                       director  \
0    Movie      

## Por último, guardar el dataframe resultante

In [2]:
df_resultante.to_csv("df_resultante.csv", index=True) 

NameError: name 'df_resultante' is not defined

In [1]:
def categorizar_por_edades(edad):
        if edad >= 60:
            return 'Senior'
        elif 29 <= edad < 60:
            return 'Adult'
        elif 15 < edad < 29:
            return 'Young adult'
        else:
            return np.nan